### im2col을 이용하여 합성곱 계층 구현해보기

In [1]:
import sys, os
import numpy as np
from collections import OrderedDict
sys.path.append(os.pardir)

from scratch.common.util import im2col
from scratch.common.layers import *


In [2]:
x1 = np.random.rand(1,3,7,7) # 데이터수, 채널수, 높이, 폭
col1 = im2col(x1, 5, 5, stride=1, pad=0) # 입력 이미지, 필터 높이, 필터 폭, 보폭, 패딩
print(col1.shape)

x2 = np.random.rand(10,3,7,7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


(연산 횟수, 필터의 넓이 * 채널수)

In [16]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = 1 + int((H + 2*self.pad - FH) / self.stride)
        out_w = 1 + int((W + 2*self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T

        out = np.dot(col, col_W) + self.b
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        self.x = x
        self.col = col
        self.col_W = col_W

        return out

    def backward(self, dout):
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)

        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol = np.dot(dout, self.col_W.T)
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)

        return dx

reshape함수에 -1사용시 전체 원소의 개수를 유지시켜 준다. 
원소수가 750일때, reshape(10, -1)를 하면 10x75로 만들어 준다.

transpose()는 축의 순서를 바꿔주는 역할을 함. (N, H, W, C) -> (N, C, H, W)

In [14]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
        self.x = None
        self.arg_max = None

    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)

        arg_max = np.argmax(col, axis=1)
        out = np.max(col, axis=1)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        self.x = x
        self.arg_max = arg_max

        return out

    def backward(self, dout):
        dout = dout.transpose(0, 2, 3, 1)
        
        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (pool_size,)) 
        
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
        
        return dx

### simple CNN 구현

input -> Conv - ReLu - Pooling -> Affine - ReLu -> Affine - Softmax -> output

* input_dim - 입력데이터(채널수, 높이, 너비)의 차원
* conv_param - 합성곱계층의 하이퍼파라미터(딕셔너리). 딕셔너리의 키는 다음과 같다
    * filter_num - 필터수
    * filter_size - 필터 폭
    * stride 
    * pad
    * hidden_size - 은닉층(완전연결)의 뉴런수
    * ouput_size - 출력층(완전연결)의 뉴런수
    * weight_init_std - 초기화시 사용할 가중치 표준편차

In [9]:
class SimpleConvNet:
    def __init__(self, input_dim=(1,28,28), conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1}, hidden_size=100, output_size=10, weight_init_std=0.01):
        
        # 초기화 인수로 주어진 하이퍼파라미터 꺼내기
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        
        # 합성곱 계층의 출력크기 계산해놓기
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad)/ filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))
        
        
        # 가중치 매개변수 초기화  
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.rand(filter_num, input_dim[0], filter_size, filter_size) # N, C, H, W
        self.params['b1'] = np.zeros(filter_num) # 편향은 각 필터 마다 하나씩 존재
        
        self.params['W2'] = weight_init_std * np.random.rand(pool_output_size, hidden_size) # 풀링 이후 사용할 Affine layer 용
        self.params['b2'] = np.zeros(hidden_size) # affine layer에서는 출력 종류만큼 편향이 존재
        self.params['W3'] = weight_init_std * np.random.rand(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        
        # CNN을 구성할 layer생성
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h = 2, pool_w = 2, stride = 2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        self.last_layer = SoftmaxWithLoss()
        
        
    def predict(self, x): # 추론
        for layer in self.layers.values():
            x = layer.forward(x)
            
        return x
    
    def loss(self, x, t): # 손실함수
        y = self.predict(x)
        
        return self.last_layer.forward(y,t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis = 1)   
       
        
        accuracy = np.sum(y==t) / float(x.shape[0])
        return accuracy
    
    def gradient(self, x, t): # 손실함수를 이용하여 매개변수를 업데이트 - 기울기는 오차역전파법 이용하여 구함
        # 순전파
        self.loss(x, t)
        
        # 역전파
        dout = 1
        dout = self.last_layer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db
        
        return grads

In [17]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
import matplotlib.pyplot as plt
from scratch.dataset.mnist import load_mnist
from scratch.common.trainer import Trainer

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

# 시간이 오래 걸릴 경우 데이터를 줄인다.
#x_train, t_train = x_train[:5000], t_train[:5000]
#x_test, t_test = x_test[:1000], t_test[:1000]

max_epochs = 20

network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
                        
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# 매개변수 보존
network.save_params("params.pkl")
print("Saved Network Parameters!")

# 그래프 그리기
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker='o', label='train', markevery=2)
plt.plot(x, trainer.test_acc_list, marker='s', label='test', markevery=2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

train loss:2.2849578868096687
=== epoch:1, train acc:0.105, test acc:0.11 ===
train loss:2.2683501541603643
train loss:2.2886386888278425
train loss:2.305874983695793
train loss:2.2972000222842848
train loss:2.286390931580401
train loss:2.282507176200648
train loss:2.2673033600893295
train loss:2.2654547808035255
train loss:2.2439238007415128
train loss:2.2712259438014413
train loss:2.2142664109099996
train loss:2.2202462221662618
train loss:2.174823813993483
train loss:2.1590938869583955
train loss:2.166312688156487
train loss:2.155129303747821
train loss:2.055112378449658
train loss:2.019792956786058
train loss:2.027178136024489
train loss:1.8981090165911325
train loss:1.9683953354412815
train loss:1.8192519180374762
train loss:1.7154023064071573
train loss:1.5172509066584707
train loss:1.71839615534341
train loss:1.7268300338849252
train loss:1.582082315229575
train loss:1.4099226181635145
train loss:1.3083962407607819
train loss:1.2416167911886173
train loss:1.2689443889380358
trai

train loss:0.25938498879928107
train loss:0.3048807944588043
train loss:0.18581875148768187
train loss:0.43746661590190705
train loss:0.39350455666081474
train loss:0.3144533634324742
train loss:0.18041899075746962
train loss:0.2304567100554762
train loss:0.23368744022381813
train loss:0.2433450439578178
train loss:0.1261311998505414
train loss:0.34228894502274115
train loss:0.3974189416511723
train loss:0.22662255208994544
train loss:0.3423176242744807
train loss:0.2773753679195867
train loss:0.3209356844430165
train loss:0.3779523365459098
train loss:0.4171169869113054
train loss:0.31711340164069396
train loss:0.45213283963720374
train loss:0.24309640467569288
train loss:0.29345190997309784
train loss:0.2748132762042452
train loss:0.32584122801153087
train loss:0.5232372330129362
train loss:0.30771300002901086
train loss:0.20438943943723792
train loss:0.40086624645504776
train loss:0.28046820677016904
train loss:0.36060863742845706
train loss:0.21456173948350124
train loss:0.19064674

train loss:0.10716636848693062
train loss:0.10569037384122991
train loss:0.14795164064647265
train loss:0.1830401156003762
train loss:0.19083652959822126
train loss:0.21018593502103147
train loss:0.2775340955241097
train loss:0.4988726994413708
train loss:0.29328199952556955
train loss:0.16260519343309318
train loss:0.20788168025926268
train loss:0.18826383658768428
train loss:0.18309082862446613
train loss:0.32663567260082
train loss:0.2564416081356835
train loss:0.1338674148368113
train loss:0.1968243496388803
train loss:0.18199464342237712
train loss:0.17069607762871095
train loss:0.2135751352910097
train loss:0.11722580235011394
train loss:0.2938106644174047
train loss:0.21416220971322467
train loss:0.06335850683168506
train loss:0.23892167123269709
train loss:0.2500638590527351
train loss:0.1670109883720162
train loss:0.11932547198202138
train loss:0.2274651445875217
train loss:0.2656051015266353
train loss:0.2166284275660556
train loss:0.2821802699451482
train loss:0.280497110534

train loss:0.17734795655742752
train loss:0.1734757668016233
train loss:0.10920582815328395
train loss:0.16527298152879577
train loss:0.10473102601155858
train loss:0.11406589208183834
train loss:0.1656472203605116
train loss:0.22766078206808182
train loss:0.25097393489970393
train loss:0.14709017203764765
train loss:0.15545977812893777
train loss:0.16000180584241416
train loss:0.1950899434679834
train loss:0.14913851897070468
train loss:0.17451399804405046
train loss:0.12293736842984071
train loss:0.175074297947842
train loss:0.13515474406051098
train loss:0.16139309802213492
train loss:0.1448084558955462
train loss:0.10206944595557052
train loss:0.1315241796641488
train loss:0.306527516791403
train loss:0.4305269166570183
train loss:0.17352328797107913
train loss:0.17577038063929723
train loss:0.13929860907667652
train loss:0.3202790500530611
train loss:0.22972785237416637
train loss:0.4027843986912817
train loss:0.26145490080218997
train loss:0.11350416174072704
train loss:0.1224558

train loss:0.12958905838143608
train loss:0.10731397097961201
train loss:0.05360976420171439
train loss:0.484529726725358
train loss:0.13482259524449988
train loss:0.2334084463013821
train loss:0.11736879857398907
train loss:0.23871509564839044
train loss:0.13566953819157124
train loss:0.05292637248837818
train loss:0.13587830643982243
train loss:0.19446045449219135
train loss:0.3646398262982517
train loss:0.08484887632757199
train loss:0.09749238507024365
train loss:0.09711356233352962
train loss:0.06359665997854297
train loss:0.16011508742834438
train loss:0.09327190568647309
train loss:0.12493277158687041
train loss:0.17514770159372348
train loss:0.07667000539266991
train loss:0.09360005954809958
train loss:0.08722723088517684
train loss:0.08138827583487986
train loss:0.11819607780952662
train loss:0.10358246031678593
train loss:0.12364604647186005
train loss:0.17191388084208295
train loss:0.15804794187494325
train loss:0.09122686799909592
train loss:0.1825423772496645
train loss:0.

...

train loss:0.0007786965499943218
train loss:0.00032510205867700685
train loss:0.0006479547516119589
train loss:0.0016225905235095243
train loss:0.0006315761732221006
train loss:0.0002018735741500712
train loss:0.0019744565650300534
train loss:2.2267217270873783e-05
train loss:0.0006074428989289133
train loss:0.0003769573390380213
train loss:0.0003352125502098584
train loss:0.004275254349597668
train loss:0.0003220237214357016
train loss:4.916100487223519e-05
train loss:0.0016372197209489458
train loss:0.0013087548338858693
train loss:0.0015871528858455192
train loss:0.00019525382175033363
train loss:0.0003826882457409034
train loss:0.0011229561707726602
train loss:5.853720640678787e-05
train loss:0.0004957446265386076
train loss:0.00031174795029435104
train loss:0.007160142849423817
train loss:0.0011710763923844542
train loss:0.0070656116763123025
train loss:0.0014610893965348029
train loss:0.0025824983551161213
train loss:0.0012345808252095395
train loss:0.0002963759492287928
train lo

train loss:0.0031185013749041156
train loss:0.0024122386170495054
train loss:0.004496858292815927
train loss:0.0055209538527999525
train loss:0.0011847374157602708
train loss:0.00012273282811950346
train loss:0.0016243911090300684
train loss:0.00025044475835024914
train loss:0.002496981288064655
train loss:0.0001492668176491404
train loss:0.0002899738943033945
train loss:0.0019249427105449588
train loss:0.00016340568380412578
train loss:0.00026363844259653436
train loss:0.00021223115522890824
train loss:0.0010023723454527802
train loss:0.004603380311812649
train loss:0.00043693852823501004
train loss:0.0013901382153834897
train loss:0.00012589062489298022
train loss:0.0009610320089752099
train loss:3.5352830921278475e-05
train loss:0.00020785933631390464
train loss:0.005602755243415258
train loss:0.0034881780820743783
train loss:0.003267958050224316
train loss:0.0001410007177142192
train loss:0.0010664296995693146
train loss:0.006393038852881124
train loss:0.003184731174641274
train lo

train loss:0.025517770273634107
train loss:0.0029715098964386783
train loss:0.0016002462789436595
train loss:0.008813136311562695
train loss:0.0001021350851559306
train loss:0.0013912490091508955
train loss:0.0009656760455354743
train loss:0.0003420800828290036
train loss:0.00129289915226527
train loss:0.002045474988969836
train loss:0.0023437144047907276
train loss:0.001206566491282998
train loss:0.0011270023118598903
train loss:0.0029232371318303274
train loss:0.001002372432089485
train loss:0.0014781083146410306
train loss:0.003067896491159489
train loss:0.0021684915318412865
train loss:0.006528472360362374
train loss:0.00024328651087538085
train loss:0.0016924171639078535
train loss:0.0001994016659009087
train loss:0.0002914347685469692
train loss:0.00014187642165393164
train loss:0.02196965036332049
train loss:0.0028311213501404538
train loss:0.024784253244862237
train loss:0.0036360405292562557
train loss:0.004125838847858797
train loss:0.0007999524190568791
train loss:0.00017345

train loss:0.004096624883858899
train loss:0.0003494410403745573
train loss:0.0019623192725324356
train loss:0.0013981980956873723
train loss:0.003134373120139277
train loss:0.0007278501189320458
train loss:0.0020738139193620473
train loss:0.0017601515848582178
train loss:0.0018438054400357529
train loss:0.0012334739861864594
train loss:0.00016206402172818838
train loss:0.0003754576424781993
train loss:0.00199479106883546
train loss:0.00018654025848562924
train loss:0.0009121437189289852
train loss:0.00013456842980286095
train loss:0.004440921370614376
train loss:0.0022084382894243196
train loss:0.0003040357597063126
train loss:0.001131607608386499
train loss:0.004110883041856681
train loss:2.063994069401151e-05
train loss:0.0046963680095169465
train loss:0.0013543229948033916
train loss:0.003086073460559702
train loss:0.02200414978857141
train loss:0.00046181525010380065
train loss:0.007532921075560937
train loss:0.0010642998407082462
train loss:0.0001438979387411871
train loss:0.0003

AttributeError: 'SimpleConvNet' object has no attribute 'save_params'